In [2]:
from gurobipy import *
import pandas as pd
import numpy as np
import csv

In [3]:
df = pd.read_excel("../data/焚化廠資料.xlsx")
df

,焚化廠名稱,縣市,年份,其他成本,勞動成本,焚化處理量,設計處理量,設計發電量,實際發電量,二氧化碳當量
0,宜蘭縣利澤垃圾資源回收(焚化)廠,宜蘭縣,2020,960330.0,1033323.0,182263.4,600,14700000,99631.87,136697.550
1,宜蘭縣利澤垃圾資源回收(焚化)廠,宜蘭縣,2011,329010.0,468548.0,199732.2,600,14700000,107838.70,149799.150
2,宜蘭縣利澤垃圾資源回收(焚化)廠,宜蘭縣,2019,846509.0,999635.0,202111.2,600,14700000,107890.40,151583.400
3,宜蘭縣利澤垃圾資源回收(焚化)廠,宜蘭縣,2012,371861.0,518953.0,202728.5,600,14700000,110156.70,152046.375
4,宜蘭縣利澤垃圾資源回收(焚化)廠,宜蘭縣,2013,411821.0,587162.0,205014.9,600,14700000,110901.60,153761.175
...,...,...,...,...,...,...,...,...,...,...
235,臺南市城西垃圾焚化廠,臺南市,2014,1085896.0,1285597.5,219334.7,900,14300000,91008.24,164501.025
236,臺南市城西垃圾焚化廠,臺南市,2016,1196041.0,1387306.0,219850.9,900,14300000,92339.52,164888.175
237,臺南市城西垃圾焚化廠,臺南市,2015,1096999.5,1299537.5,221728.8,900,14300000,93911.04,166296.600
238,臺南市城西垃圾焚化廠,臺南市,2011,835958.0,999123.0,222657.9,900,14300000,91025.38,166993.425


In [4]:
df.describe()

,年份,其他成本,勞動成本,焚化處理量,設計處理量,設計發電量,實際發電量,二氧化碳當量
count,240.000000,2.400000e+02,2.400000e+02,240.000000,240.000000,2.400000e+02,240.000000,240.000000
mean,2015.500000,1.153994e+06,1.564774e+06,264436.511708,1027.083333,2.311321e+07,133369.255292,198327.383781
std,2.878284,7.265852e+05,8.473039e+05,90906.892564,371.860404,1.154901e+07,62467.998158,68180.169423
min,2011.000000,9.946700e+04,4.506970e+05,63239.450000,300.000000,2.326000e+06,15989.940000,47429.587500
25%,2013.000000,7.192419e+05,1.009241e+06,199559.525000,900.000000,1.452500e+07,90917.182500,149669.643750
50%,2015.500000,9.426180e+05,1.307989e+06,253034.050000,900.000000,2.255000e+07,132502.500000,189775.537500
75%,2018.000000,1.750400e+06,2.206284e+06,303204.325000,1350.000000,2.790000e+07,170481.750000,227403.243750
max,2020.000000,3.730386e+06,5.417133e+06,472580.700000,1800.000000,4.900000e+07,270666.500000,354435.525000


In [24]:
# Sets
Years = list(range(2011, 2021))
K = list(range(24))  # number of DMUs
I_V = [0,1,2]  # variable input amount
I_F = [0,1]  # fixed input amount
J = [0]  # output 
M = [0]  # bad output
years = list(range(1,11))
dmus = list(range(1,25))

In [29]:
yearly_df = df.groupby('年份')

In [32]:
yearly_df.get_group(2020)["其他成本"]

0      9.603300e+05
17     1.308329e+06
20     8.950710e+05
30     3.730386e+06
40     7.715248e+05
56     7.715248e+05
63     7.715248e+05
75     7.715248e+05
80     2.872190e+05
92     2.569322e+06
107    2.569322e+06
110    2.569322e+06
121    4.739010e+05
130    3.417060e+05
140    6.390130e+05
150    1.381022e+06
160    1.502592e+06
170    1.502592e+06
180    1.502592e+06
190    2.833317e+05
202    2.833317e+05
211    2.833317e+05
220    1.090514e+06
230    1.090514e+06
Name: 其他成本, dtype: float64

In [34]:
year = 2020
# Parameters
X_V = [
    list(yearly_df.get_group(year)["其他成本"].values),
    list(yearly_df.get_group(year)["勞動成本"].values),
    list(yearly_df.get_group(year)["焚化處理量"].values)
]
X_F = [
    list(yearly_df.get_group(year)["設計處理量"].values),
    list(yearly_df.get_group(year)["設計發電量"].values)
]
Y = [list(yearly_df.get_group(year)["實際發電量"].values)]
B = [list(yearly_df.get_group(year)["二氧化碳當量"].values)]

In [35]:
crs = [0 for r in K]

for r in K:
    md11 = Model('model')
    l = md11.addVars(K, vtype=GRB.CONTINUOUS, name='l')
    x = md11.addVars(I_V, vtype=GRB.CONTINUOUS, name='x')
    y = md11.addVars(J, vtype=GRB.CONTINUOUS, name='y')  

    md11.setObjective(
        quicksum(l[k] * B[0][k] for k in K), GRB.MAXIMIZE
    )

    md11.addConstrs(
        quicksum(l[k] * X_F[i][k] for k in K) <= X_F[i][r]
        for i in I_F
    )

    md11.addConstrs(
        quicksum(l[k] * X_V[i][k] for k in K) <= x[i]
        for i in I_V
    )

    md11.addConstrs(
        quicksum(l[k] * Y[j][k] for k in K) >= y[j]
        for j in J
    )

    md11.params.LogToConsole = 0
    # md11.params.NonConvex = 2
    md11.optimize() 

    print(df.loc[r,"焚化廠名稱"])
    for var in md11.getVars():
        print(f"{var.VarName}, {var.X}")

    # to load crs
    crs[r] = md11.objVal

宜蘭縣利澤垃圾資源回收(焚化)廠
l[0], 1.0
l[1], 0.0
l[2], 0.0
l[3], 0.0
l[4], 0.0
l[5], 0.0
l[6], 0.0
l[7], 0.0
l[8], 0.0
l[9], 0.0
l[10], 0.0
l[11], 0.0
l[12], 0.0
l[13], 0.0
l[14], 0.0
l[15], 0.0
l[16], 0.0
l[17], 0.0
l[18], 0.0
l[19], 0.0
l[20], 0.0
l[21], 0.0
l[22], 0.0
l[23], 0.0
x[0], 960330.0
x[1], 1033323.0
x[2], 182263.4
y[0], 0.0
宜蘭縣利澤垃圾資源回收(焚化)廠
l[0], 1.5
l[1], 0.0
l[2], 0.0
l[3], 0.0
l[4], 0.0
l[5], 0.0
l[6], 0.0
l[7], 0.0
l[8], 0.0
l[9], 0.0
l[10], 0.0
l[11], 0.0
l[12], 0.0
l[13], 0.0
l[14], 0.0
l[15], 0.0
l[16], 0.0
l[17], 0.0
l[18], 0.0
l[19], 0.0
l[20], 0.0
l[21], 0.0
l[22], 0.0
l[23], 0.0
x[0], 1440495.0
x[1], 1549984.5
x[2], 273395.1
y[0], 0.0
宜蘭縣利澤垃圾資源回收(焚化)廠
l[0], 0.7885483014861996
l[1], 0.0
l[2], 0.0
l[3], 0.0
l[4], 0.0
l[5], 0.0
l[6], 0.0
l[7], 0.0
l[8], 0.0
l[9], 0.0
l[10], 0.0
l[11], 0.0
l[12], 0.0
l[13], 0.08957006369426741
l[14], 0.0
l[15], 0.0
l[16], 0.0
l[17], 0.0
l[18], 0.0
l[19], 0.0
l[20], 0.0
l[21], 0.0
l[22], 0.0
l[23], 0.0
x[0], 787873.2185509554
x[1], 884349.648686

B_minus 都是負數：表示每間焚化廠說不定可以多生產一些碳

In [37]:
crs

[136697.55,
 205046.32499999998,
 112040.89204319267,
 307569.4875,
 307569.4875,
 307569.4875,
 205046.32499999998,
 410092.64999999997,
 136697.55,
 307569.4875,
 174025.63493730093,
 262703.99281200237,
 205046.32499999998,
 47429.587499999994,
 205046.32499999998,
 205046.32499999998,
 167363.4733130971,
 205046.32499999998,
 205046.32499999998,
 122346.31341358555,
 244934.33889828817,
 410092.64999999997,
 205046.32499999998,
 172776.47963276273]

In [52]:
sum(B[0])

4421411.2875

In [152]:
X_V[0]

[960330.0,
 1308329.0,
 895071.0,
 3730386.0,
 771524.75,
 771524.75,
 771524.75,
 771524.75,
 287219.0,
 2569321.6666666665,
 2569321.6666666665,
 2569321.6666666665,
 473901.0,
 341706.0,
 639013.0,
 1381022.0,
 1502592.3333333333,
 1502592.3333333333,
 1502592.3333333333,
 283331.6666666667,
 283331.6666666667,
 283331.6666666667,
 1090514.5,
 1090514.5]

In [86]:
np.array(B[0])

array([136697.55  , 188625.6   , 109951.725 , 289382.475 , 284787.75  ,
       245548.35  , 140791.725 , 282796.275 , 133405.125 , 284376.525 ,
       151773.825 , 188916.675 , 164930.1   ,  47429.5875, 201153.15  ,
       190617.525 , 136925.625 , 196591.8   , 203855.325 , 101225.1   ,
       162133.8   , 267403.5   , 188394.525 , 123697.65  ])

In [165]:
B_minus = (np.array(B[0])- np.array(crs)).tolist()  #減碳下限
# B_minus = [0 for k in K]
B_plus = B[0]   #減碳上限為原本的排放量
C = sum(B[0]) * 0.01    #Cap of trade
D = 2000000  #Demand of electricity
# P_X = 590
P_Y = 4 * 1e7
tao = 123
Y_bar = 10975
epsolon = 0

## 計算碳排放權的分配

In [166]:
# Sets
Year = list(range(2011, 2021))
K = list(range(24))  # number of DMUs
I_V = [0,1]  # variable input amount
I_F = [0,1]  # fixed input amount
J = [0]  # output 
M = [0]  # bad output
years = list(range(1,10))
dmus = list(range(1,25))

In [173]:
revenue = [0 for k in K]

md2 = Model('model')
BOUND = 0.00000001
LB = -GRB.INFINITY
_lambda = md2.addVars(K, K, vtype=GRB.CONTINUOUS, name='lambda')
x = md2.addVars(I_V, K, vtype=GRB.CONTINUOUS, name='x')
y = md2.addVars(K, vtype=GRB.CONTINUOUS, name='y') 
b = md2.addVars(K, vtype=GRB.CONTINUOUS, name='b', lb=LB) 
phi1 = md2.addVars(I_F, K, vtype=GRB.CONTINUOUS, name='phi1', lb=LB)
phi2 = md2.addVars(I_V, K, vtype=GRB.CONTINUOUS, name='phi2', lb=LB)
phi3 = md2.addVars(K, vtype=GRB.CONTINUOUS, name='phi3', lb=LB)
phi4 = md2.addVars(K, vtype=GRB.CONTINUOUS, name='phi4', lb=LB)
phi5 = md2.addVar(vtype=GRB.CONTINUOUS, name='phi5', lb=LB)
phi6 = md2.addVar(vtype=GRB.CONTINUOUS, name='phi6', lb=LB)
phi7 = md2.addVars(K, vtype=GRB.CONTINUOUS, name='phi7', lb=LB)
phi8 = md2.addVars(K, vtype=GRB.CONTINUOUS, name='phi8', lb=LB)
a1 = md2.addVars(I_V, K, vtype=GRB.CONTINUOUS, name='a1')
a2 = md2.addVars(K, vtype=GRB.CONTINUOUS, name='a2')
a3 = md2.addVars(K, vtype=GRB.CONTINUOUS, name='a3')
a4 = md2.addVars(K, K, vtype=GRB.CONTINUOUS, name='a4')
a5 = md2.addVars(I_F, K, vtype=GRB.CONTINUOUS, name='a5')
a6 = md2.addVars(I_V, K, vtype=GRB.CONTINUOUS, name='a6')
a7 = md2.addVars(K, vtype=GRB.CONTINUOUS, name='a7')
a8 = md2.addVars(K, vtype=GRB.CONTINUOUS, name='a8')
a9 = md2.addVar(vtype=GRB.CONTINUOUS, name='a9')
a10 = md2.addVar(vtype=GRB.CONTINUOUS, name='a10')
a11 = md2.addVars(K, vtype=GRB.CONTINUOUS, name='a11')
a12 = md2.addVars(K, vtype=GRB.CONTINUOUS, name='a12')

md2.setObjective(
    quicksum(a1[i, r] for i in I_V for r in K) 
    + quicksum(a2[r] for r in K)
    + quicksum(a3[r] for r in K)
    + quicksum(a4[k, r] for k in K for r in K)
    + quicksum(a5[i, r] for i in I_F for r in K)
    + quicksum(a6[i, r] for i in I_V for r in K)
    + quicksum(a7[r] for r in K)
    + quicksum(a8[r] for r in K)
    + a9 +a10
    + quicksum(a11[r] for r in K)
    + quicksum(a12[r] for r in K)
    , GRB.MINIMIZE
)

## KKT 
md2.addConstrs(
    x[i,r] * (
        1 - (phi2[i,r]) 
    ) - a1[i, r] == 0
    for i in I_V for r in K
)

md2.addConstrs(
    y[r] *
    (P_Y - tao * (
        quicksum(y[k] for k in K) + Y_bar   #Y_hat
    ) - tao * y[r] - phi3[r] + phi5) 
    - a2[r] == 0
    for r in K
)

md2.addConstrs(
    epsolon - phi4[r] + phi6 + phi7[r] - phi8[r] 
    - a3[r] == 0
    for r in K
)

md2.addConstrs(
    _lambda[k, r] * 
    (
        - quicksum(phi1[i,r] * X_F[i][k] for i in I_F)  
        - quicksum(phi2[i,r] * X_V[i][k] for i in I_V)
        + phi3[r] * Y[0][k]
        - phi4[r] * B[0][k]
    ) - a4[k, r] == 0
    for r in K for k in K
)

md2.addConstrs(
    phi1[i,r] * 
    ( 
        quicksum(_lambda[k,r] * X_F[i][k] for k in K) - X_F[i][r] 
    ) - a5[i, r] == 0
    for i in I_F for r in K
)

md2.addConstrs(
    phi2[i,r] * 
    ( 
        quicksum(_lambda[k,r] * X_V[i][k] for k in K) - x[i,r] 
    ) - a6[i, r] == 0
    for i in I_V for r in K
)

md2.addConstrs(
    phi3[r] * 
    ( 
        y[r] - quicksum(_lambda[k,r] * Y[0][k] for k in K) 
    ) - a7[r] == 0
    for r in K
)

md2.addConstrs(
    quicksum(_lambda[k,r] * B[0][k] for k in K) - B[0][r] + b[r]
    - a8[r] == 0
    for r in K
)

md2.addConstr(
    phi5 *
    (D - quicksum(y[r] for r in K) )
    - a9 == 0
)

md2.addConstr(
    C - quicksum(b[r] for r in K) - a10 == 0
)

md2.addConstrs(
    phi7[r] * 
    ( 
        B_minus[r] - b[r]
    ) - a11[r] == 0
    for r in K
)

md2.addConstrs(
    phi8[r] *
    ( 
        b[r] - B_plus[r]
    ) - a12[r] == 0
    for r in K
)


md2.write("1.lp")
# md2.params.LogToConsole = 0
md2.params.NonConvex = 2
md2.optimize() 

for var in md2.getVars():
    print(f"{var.VarName}, {var.X}")

csv_file = open("b_minus_is_0_constraint_equal0.csv", "a")
for var in md2.getVars(): 
    writer = csv.writer(csv_file)
    writer.writerow([var.VarName, var.X])
csv_file.close()


Set parameter NonConvex to value 2
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 697 rows, 1732 columns and 5545 nonzeros
Model fingerprint: 0x6b75f8f1
Model has 169 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 5e+07]
  QMatrix range    [1e+00, 5e+07]
  QLMatrix range   [1e+00, 5e+07]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+07]

Continuous model is non-convex -- solving as a MIP

Presolve removed 48 rows and 648 columns
Presolve time: 0.03s
Presolved: 12866 rows, 4061 columns, 32187 nonzeros
Presolved model has 3024 bilinear constraint(s)
Variable types: 4061 continuous, 0 integer (0 binary)

Root relaxation: objective 4.800000e+01, 5255 iterations, 0.07 seconds (0.13 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf 

In [ ]:
# revenue = [0 for k in K]

# md2 = Model('model')
# BOUND = 0.00000001
# LB = -GRB.INFINITY
# _lambda = md2.addVars(K, K, vtype=GRB.CONTINUOUS, name='lambda')
# x = md2.addVars(I_V, K, vtype=GRB.CONTINUOUS, name='x')
# y = md2.addVars(K, vtype=GRB.CONTINUOUS, name='y') 
# b = md2.addVars(K, vtype=GRB.CONTINUOUS, name='b', lb=LB) 
# phi1 = md2.addVars(I_F, K, vtype=GRB.CONTINUOUS, name='phi1', lb=LB)
# phi2 = md2.addVars(I_V, K, vtype=GRB.CONTINUOUS, name='phi2', lb=LB)
# phi3 = md2.addVars(K, vtype=GRB.CONTINUOUS, name='phi3', lb=LB)
# phi4 = md2.addVars(K, vtype=GRB.CONTINUOUS, name='phi4', lb=LB)
# phi5 = md2.addVar(vtype=GRB.CONTINUOUS, name='phi5', lb=LB)
# phi6 = md2.addVar(vtype=GRB.CONTINUOUS, name='phi6', lb=LB)
# phi7 = md2.addVars(K, vtype=GRB.CONTINUOUS, name='phi7', lb=LB)
# phi8 = md2.addVars(K, vtype=GRB.CONTINUOUS, name='phi8', lb=LB)
# a1 = md2.addVars(I_V, K, vtype=GRB.CONTINUOUS, name='a1', lb=LB )
# a2 = md2.addVars(K, vtype=GRB.CONTINUOUS, name='a2', lb=LB)
# a3 = md2.addVars(K, vtype=GRB.CONTINUOUS, name='a3', lb=LB)
# a4 = md2.addVars(K, K, vtype=GRB.CONTINUOUS, name='a4', lb=LB)
# a5 = md2.addVars(I_F, K, vtype=GRB.CONTINUOUS, name='a5', lb=LB)
# a6 = md2.addVars(I_V, K, vtype=GRB.CONTINUOUS, name='a6', lb=LB)
# a7 = md2.addVars(K, vtype=GRB.CONTINUOUS, name='a7', lb=LB)
# a8 = md2.addVars(K, vtype=GRB.CONTINUOUS, name='a8', lb=LB)
# a9 = md2.addVar(vtype=GRB.CONTINUOUS, name='a9', lb=LB)
# a10 = md2.addVar(vtype=GRB.CONTINUOUS, name='a10', lb=LB)
# a11 = md2.addVars(K, vtype=GRB.CONTINUOUS, name='a11', lb=LB)
# a12 = md2.addVars(K, vtype=GRB.CONTINUOUS, name='a12', lb=LB)


# # md2.setObjective(
# #     quicksum(
# #         y[r] * (P_Y - tao * (
# #             quicksum(y[k] for k in K) + Y_bar
# #         )) - quicksum(x[i, r] for i in I_V)
# #         - epsolon * (B[0][r] - b[r])
# #         - quicksum(phi1[i,r] * (quicksum(l[k,r] * X_F[i][k] for k in K) - X_F[i][r]) for i in I_F )
# #         - quicksum(phi2[i,r] * (quicksum(l[k,r] * X_V[i][k] for k in K) - x[i,r]) for i in I_V )
# #         - phi3[r] * (y[r] - quicksum( l[k,r] * Y[0][k] for k in K))
# #         - phi4[r] * (quicksum(l[k,r] * B[0][k] for k in K) - B[0][r] + b[r] )
# #         - phi5 * (D - quicksum(y[r] for r in K))
# #         - phi6 * (C - quicksum(b[r] for r in K))
# #         - phi7[r] * (B_minus[r] - b[r])
# #         - phi8[r] * (b[r] - B_plus[r])
# #     for r in K)
# #     ,  GRB.MAXIMIZE
# # )

# md2.setObjective(
#     quicksum(a1[i, r] for i in I_V for r in K) 
#     + quicksum(a2[r] for r in K)
#     + quicksum(a3[r] for r in K)
#     + quicksum(a4[k, r] for k in K for r in K)
#     + quicksum(a5[i, r] for i in I_F for r in K)
#     + quicksum(a6[i, r] for i in I_V for r in K)
#     + quicksum(a7[r] for r in K)
#     + quicksum(a8[r] for r in K)
#     + a9 + a10
#     + quicksum(a11[r] for r in K)
#     + quicksum(a12[r] for r in K)
#     , GRB.MINIMIZE
# )

# ## Original Constraints
# # md2.addConstrs(
# #     quicksum(l[k,r] * X_F[i][k] for k in K) <= X_F[i][r]
# #     for i in I_F
# # )

# # md2.addConstrs(
# #     quicksum(l[k,r] * X_V[i][k] for k in K) <= x[i, r]
# #     for i in I_V
# # )

# # md2.addConstr(
# #     quicksum(l[k,r] * Y[0][k] for k in K) >= y[r]
# # )

# # md2.addConstr(
# #     quicksum(l[k,r] * B[0][k] for k in K) == B[0][r] - b[r]
# # )

# # md2.addConstr(
# #     quicksum(y[r] for r in K) >= D
# # )

# # md2.addConstr(
# #     quicksum(b[r] for r in K) == C 
# # )

# # md2.addConstr(
# #     b[r] >= B_minus[r]
# # )

# # md2.addConstr(
# #     b[r] <= B_plus[r]
# # )


# ## KKT 
# md2.addConstrs(
#     1 - (phi2[i,r]) - a1[i, r] <= BOUND
#     for i in I_V for r in K
# )

# md2.addConstrs(
#     (P_Y - tao * (
#         quicksum(y[k] for k in K) + Y_bar   #Y_hat
#     ) - tao * y[r] - phi3[r] + phi5) 
#     - a2[r] <= BOUND
#     for r in K
# )

# md2.addConstrs(
#     epsolon - phi4[r] + phi6 + phi7[r] - phi8[r] 
#     - a3[r] <= BOUND
#     for r in K
# )

# md2.addConstrs(
#     (
#         - quicksum(phi1[i,r] * X_F[i][k] for i in I_F)  
#         - quicksum(phi2[i,r] * X_V[i][k] for i in I_V)
#         + phi3[r] * Y[0][k]
#         - phi4[r] * B[0][k]
#     ) - a4[k, r] <= BOUND
#     for r in K for k in K
# )

# md2.addConstrs(
#     ( 
#         quicksum(_lambda[k,r] * X_F[i][k] for k in K) - X_F[i][r] 
#     ) - a5[i, r] <= BOUND
#     for i in I_F for r in K
# )

# md2.addConstrs(
#     ( 
#         quicksum(_lambda[k,r] * X_V[i][k] for k in K) - x[i,r] 
#     ) - a6[i, r] <= BOUND
#     for i in I_V for r in K
# )

# md2.addConstrs(
#     ( 
#         y[r] - quicksum(_lambda[k,r] * Y[0][k] for k in K) 
#     ) - a7[r] <= BOUND
#     for r in K
# )

# md2.addConstrs(
#     quicksum(_lambda[k,r] * B[0][k] for k in K) - B[0][r] + b[r]
#     - a8[r] <= BOUND
#     for r in K
# )

# md2.addConstr(
#     D - quicksum(y[r] for r in K) 
#     - a9 <= BOUND
# )

# md2.addConstr(
#     C - quicksum(b[r] for r in K) - a10 <= BOUND
# )

# md2.addConstrs(
#     ( 
#         B_minus[r] - b[r]
#     ) - a11[r] <= BOUND
#     for r in K
# )

# md2.addConstrs(
#     ( 
#         b[r] - B_plus[r]
#     ) - a12[r] <= BOUND
#     for r in K
# )

# ## =============

# md2.addConstrs(
#     1 - (phi2[i,r]) - a1[i, r] >= -BOUND
#     for i in I_V for r in K
# )

# md2.addConstrs(
#     (P_Y - tao * (
#         quicksum(y[k] for k in K) + Y_bar   #Y_hat
#     ) - tao * y[r] - phi3[r] + phi5) 
#     - a2[r] >= -BOUND
#     for r in K
# )

# md2.addConstrs(
#     epsolon - phi4[r] + phi6 + phi7[r] - phi8[r] 
#     - a3[r] <= BOUND
#     for r in K
# )

# md2.addConstrs(
#     (
#         - quicksum(phi1[i,r] * X_F[i][k] for i in I_F)  
#         - quicksum(phi2[i,r] * X_V[i][k] for i in I_V)
#         + phi3[r] * Y[0][k]
#         - phi4[r] * B[0][k]
#     ) - a4[k, r] >= -BOUND
#     for r in K for k in K
# )

# md2.addConstrs(
#     ( 
#         quicksum(_lambda[k,r] * X_F[i][k] for k in K) - X_F[i][r] 
#     ) - a5[i, r] >= -BOUND
#     for i in I_F for r in K
# )

# md2.addConstrs(
#     ( 
#         quicksum(_lambda[k,r] * X_V[i][k] for k in K) - x[i,r] 
#     ) - a6[i, r] >= -BOUND
#     for i in I_V for r in K
# )

# md2.addConstrs(
#     ( 
#         y[r] - quicksum(_lambda[k,r] * Y[0][k] for k in K) 
#     ) - a7[r] >= -BOUND
#     for r in K
# )

# md2.addConstrs(
#     quicksum(_lambda[k,r] * B[0][k] for k in K) - B[0][r] + b[r]
#     - a8[r] >= -BOUND
#     for r in K
# )

# md2.addConstr(
#     D - quicksum(y[r] for r in K) 
#     - a9 >= -BOUND
# )

# md2.addConstr(
#     C - quicksum(b[r] for r in K) - a10 >= -BOUND
# )

# md2.addConstrs(
#     ( 
#         B_minus[r] - b[r]
#     ) - a11[r] >= -BOUND
#     for r in K
# )

# md2.addConstrs(
#     ( 
#         b[r] - B_plus[r]
#     ) - a12[r] >= -BOUND
#     for r in K
# )

# md2.write("1.lp")
# # md2.params.LogToConsole = 0
# # md2.params.NonConvex = 2
# md2.optimize() 

# for var in md2.getVars():
#     print(f"{var.VarName}, {var.X}")

# csv_file = open("b_minus_is_0_constraint_equal0.csv", "a")
# for var in md2.getVars(): 
#     writer = csv.writer(csv_file)
#     writer.writerow([var.VarName, var.X])
# csv_file.close()


In [78]:
ans = pd.read_csv("result.csv")
ans

,"l[0,0]",0.0
0,"l[0,1]",0.000000
1,"l[0,2]",0.000000
2,"l[0,3]",2.116954
3,"l[0,4]",0.000000
4,"l[0,5]",0.000000
...,...,...
3458,a12[19],0.000000
3459,a12[20],0.000000
3460,a12[21],0.000000
3461,a12[22],0.000000
